In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

import complexPyTorch as ctorch
from complexPyTorch import complexFunctions, complexLayers
from complexPyTorch.complexFunctions import complex_tanh

plt.rcParams['text.usetex']=True

delta=0.01
mass=1

In [2]:
class BosonWaveFunction(nn.Module):
    def __init__(self):
        super(BosonWaveFunction, self).__init__()

        self.layer1=nn.Linear(1,32)
        self.layer2=nn.Linear(32,32)
        self.layer3=nn.Linear(32,1)

        self.norm = torch.ones(1, requires_grad=False)

    def forward(self, inputs):
        x = inputs
        x = self.layer1(x)
        x = torch.tanh(x)
        x = self.layer2(x)
        x = torch.tanh(x)
        x = self.layer3(x)

        # boundary condition - normalization?
        # boundary_condition = self.exp(inputs)
        # output = self.norm*x*boundary_condition
        
        x = torch.mul(x,self.norm)

        return x
    
    def update_norm(self, inputs):
        with torch.no_grad():
            value = self.forward(inputs)
            N = value**2
            delta=inputs[1]-inputs[0]
            N=torch.sum(N)*delta 
            self.norm = self.norm*1.0/torch.sqrt(N)

In [3]:
# This is preparing input data that is differentiable:
_x = np.arange(-10, 10, delta, dtype=np.float32)
_x = _x.reshape((_x.shape[0], 1))
x = torch.tensor(_x, requires_grad=True)

In [4]:
psi = BosonWaveFunction()
print(psi.norm)
psi.update_norm(x)
print(psi.norm)
psi_y = psi(x)

tensor([1.])
tensor([0.7682])


In [35]:
def potential_energy(wf, inputs, delta):
    phi = wf(inputs)

    t1 = phi**2
    t2 = phi*(phi.roll(1)-phi.roll(-1))
    return (delta/2.0)*torch.sum(t1)+(1.0/2.0)*torch.sum(t2)

def kinetic_energy(wf, inputs, delta):
    phi = wf(inputs)
    dPhidx = torch.autograd.grad(phi, inputs=inputs, 
                                 grad_outputs=torch.ones_like(phi), retain_graph=True)

    t1 = dPhidx[0]**2/(2.0*delta)
    #t2 = ((phi.roll(1)-phi.roll(-1))/(2.0*delta))**2
    t2 = phi**2

    return torch.sum(t1) + (delta/2.0)*torch.sum(t2)

def total_energy(wf, inputs, delta):
    pe = potential_energy(wf, inputs, delta)
    ke = kinetic_energy(wf, inputs, delta)
    #print("norm=",norm,"ke=",ke,"pe=",pe)
    return (pe+ke)

In [36]:
total_energy(psi, x, delta)

tensor(353.8633, grad_fn=<AddBackward0>)

In [37]:
psi = BosonWaveFunction()
psi.update_norm(x)
psi_y = psi(x)
optimizer = torch.optim.Adam(psi.parameters(), lr=0.0001)
optimizer.zero_grad()
# cant go up to 10000?
for i in range(3000):
    psi.update_norm(x)
    energy=total_energy(psi,x,delta) # this is the loss!
    energy.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i%300==0:
        print(energy)

tensor(168.3424, grad_fn=<AddBackward0>)
tensor(765.1233, grad_fn=<AddBackward0>)


KeyboardInterrupt: 